In [50]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
import streamlit as st

## Load Data

In [2]:
df = pd.read_parquet("/Users/yiukitcheung/Documents/Projects/Stocks/train_data_repository/train_data.parquet")
df

,high,close_t-1,low,MACD_HIST,open,close_t-2,BodyDiff,volume,MACD,169EMA,...,Incremental_High_1.0,dual_channel_Alert_0,dual_channel_Alert_1,CandleStickType_red,Engulf_Alert_0,Engulf_Alert_1,MACD_Alert_0,MACD_Alert_1,timestamp,log_daily_return
0,21.388277,21.076815,20.914093,-0.230128,21.380292,21.862467,0.170706,204688000,0.256492,17.608463,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0,NaN
1,21.922363,21.209585,21.159673,-0.195934,21.329381,21.076815,0.574015,268726000,0.241703,17.658991,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.218851
2,22.496379,21.903397,21.852484,-0.133754,22.102055,21.209585,0.341415,248555000,0.270444,17.715279,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2,2.435798
3,22.111038,22.443470,21.823532,-0.117002,22.032173,21.903397,0.010981,217655000,0.257946,17.766195,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,-1.799764
4,21.761641,22.043154,21.288454,-0.130608,21.672793,22.443470,0.049914,245215000,0.211688,17.811568,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,4,-1.925010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,84.067903,82.409180,79.169722,-1.257190,83.935921,79.504669,4.272279,512208000,-1.361822,64.130016,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,651,-3.388353
652,83.309027,79.663643,78.209884,-0.975734,78.854772,82.409180,3.763376,424641000,-1.324299,64.347523,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,652,3.641607
653,88.316192,82.618149,83.373021,-0.419388,83.803948,79.663643,3.916343,551011000,-0.872800,64.622497,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,653,5.992387
654,87.977240,87.720291,85.251699,-0.043159,87.580311,82.618149,0.161975,388971000,-0.507361,64.894494,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,654,0.025071


## Support Resistance Detection

#### Fratal Candles Patterns
##### Credit : https://towardsdatascience.com/detection-of-price-support-and-resistance-levels-in-python-baedc44c34c9
Definition: 

A fractal is a candlestick pattern made by 5 candles. The third candle has the lowest low price, the previous candles have decreasing lows and the next candles have increasing lows. By this pattern, the low of the third candle is the support level. The same concept applies to resistance levels, where the third candle has the highest high of the five ones.

![Fractals Pattern](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*xOLF6WDrcvVks7CEMIVoGw.png)

### Compute Support and Resistance

* Rolling 5 days and compute the low and high
* Center Rolling Required

In [26]:
# Define the support and resistance levels
support = df[df['low'] == df['low'].rolling(5,center=True).min()][['low']]
resistance = df[df['high'] == df['high'].rolling(5,center=True).max()][['high']]


### Extract the strong Support and Resistance

* Define base on resistance/support frequency
* Set a threshold and window to define 

In [52]:
# Streamlit app
st.title("Support and Resistance Level Analysis")

# Plotting the histograms
fig = go.Figure()

# Add histogram for support level
support_hist = np.histogram(support['low'], bins=np.arange(support['low'].min(), support['low'].max() + 1, 1))
fig.add_trace(go.Histogram(
    x=support['low'], 
    name='Support Level',
    opacity=0.5,
    xbins=dict(
        start=support['low'].min(),
        end=support['low'].max(),  
        size=1
    )
))

# Add histogram for resistance level
resistance_hist = np.histogram(resistance['high'], bins=np.arange(resistance['high'].min(), resistance['high'].max() + 1, 1))
fig.add_trace(go.Histogram(
    x=resistance['high'],
    name='Resistance Level',
    opacity=0.5,
    xbins=dict(
        start=resistance['high'].min(),
        end=resistance['high'].max(),  
        size=1
    )
))

# Calculate the most frequent bin for support level
most_frequent_support_bin = support_hist[1][np.argmax(support_hist[0])]
most_frequent_support_count = np.max(support_hist[0])

# Calculate the most frequent bin for resistance level
most_frequent_resistance_bin = resistance_hist[1][np.argmax(resistance_hist[0])]
most_frequent_resistance_count = np.max(resistance_hist[0])

# Add bar to highlight the most frequent support bin
fig.add_trace(go.Bar(
    x=[most_frequent_support_bin],
    y=[most_frequent_support_count],
    name='Most Frequent Support',
    marker=dict(color='red', opacity=0.5)
))

# Add bar to highlight the most frequent resistance bin
fig.add_trace(go.Bar(
    x=[most_frequent_resistance_bin],
    y=[most_frequent_resistance_count],
    name='Most Frequent Resistance',
    marker=dict(color='blue', opacity=0.5)
))

# Update layout to overlay histograms
fig.update_layout(barmode='overlay')

# Display the plot in Streamlit
st.plotly_chart(fig)

DeltaGenerator()